In [511]:
'''
CREATE TABLE colabdust (
  num int(11) NOT NULL AUTO_INCREMENT,
  seq varchar(50) NOT NULL,
  mdate varchar(100) NOT NULL,
  area01 varchar(255) NOT NULL,
  area02 varchar(255) NOT NULL,
  so2 varchar(255) NOT NULL,
  co text NOT NULL,
  o3 varchar(255) NOT NULL,
  no2 varchar(255) NOT NULL,
  pm10 varchar(255) NOT NULL,
  pm25 varchar(255) NOT NULL,
  posted datetime DEFAULT NOW(),
  primary key (num)
) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;
END'''

'\nCREATE TABLE colabdust (\n  num int(11) NOT NULL AUTO_INCREMENT,\n  seq varchar(50) NOT NULL,\n  mdate varchar(100) NOT NULL,\n  area01 varchar(255) NOT NULL,\n  area02 varchar(255) NOT NULL,\n  so2 varchar(255) NOT NULL,\n  co text NOT NULL,\n  o3 varchar(255) NOT NULL,\n  no2 varchar(255) NOT NULL,\n  pm10 varchar(255) NOT NULL,\n  pm25 varchar(255) NOT NULL,\n  posted datetime DEFAULT NOW(),\n  primary key (num)\n) ENGINE=InnoDB AUTO_INCREMENT=4 DEFAULT CHARSET=utf8;\nEND'

In [512]:
!sudo pip3 install pymysql

In [513]:
import pymysql
import requests
import configparser
from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import drive
from xml.etree import ElementTree as et

In [514]:
c=drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [515]:
RST_URI = 'drive/My Drive/development/html/result.xml'
ERR_URI = 'drive/My Drive/development/test/error.log'
CNF_INI = 'drive/My Drive/development/config/openapi.ini'

In [516]:
config = configparser.ConfigParser()
config.read(CNF_INI)
HOST = config['mysql']['host']
USER = config['mysql']['user']
PASSWORD = config['mysql']['password']
DATABASE = config['mysql']['database']
PORT = config['mysql']['port']
CHARSET = config['mysql']['charset']
URL = config['airkorea']['url']
SERVICEKEY = config['airkorea']['servicekey']

In [517]:
def change_keyword(string):
  string = string.replace('@', '%')  
  return string  

In [518]:
def make_format(string):
  string = string.replace('-','')
  string = string.replace(':','')
  string = string.replace(' ','')		  

  return string

In [519]:
def exec_select(conn,seq,area01):
  with conn.cursor() as cursor:
    query = get_select(seq,area01)
    cursor.execute(query)
    for row in cursor:
      data = row.get('cnt')
      
  return data

In [520]:
def exec_insert(conn,seq,mdate,area01,area02,so2,co,o3,no2,pm10,pm25):
  query = get_insert(seq,mdate,area01,area02,so2,co,o3,no2,pm10,pm25)
  with conn.cursor() as cursor:
    cursor.execute(query)
  conn.commit()

In [521]:
def get_select(seq,area02):
  query = 'select \
  count(*) as cnt \
  from colabdust \
  where seq=\'{}\' and area01=\'{}\''.format(seq,area02)
  print(query)
  return query

In [522]:
def get_insert(seq,mdate,area01,area02,so2,co,o3,no2,pm10,pm25):
  query = 'insert into colabdust (seq,mdate,area01,area02,so2,co,o3,no2,pm10,pm25) \
  values (\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\')'.format(seq,mdate,area01,area02,so2,co,o3,no2,pm10,pm25)
  print(query)
  return query

In [523]:
def data_collection(sidoName, result):
  try:
    conn = pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DATABASE,
        charset=CHARSET,
        port=int(PORT),
        cursorclass=pymysql.cursors.DictCursor)

    dtime, area, so2, co, o3, no2, pm10, pm25 = [], [], [], [], [], [], [], []

    tree = et.parse(result)
    root = tree.getroot()

    for response in root:
      for body in response:
        for items in body:
          for item in items:
            if item.tag == 'dataTime':
              # print(item.text)
              dtime.append(item.text)
            elif item.tag == 'cityName':
              # print(item.text)
              area.append(item.text)
            elif item.tag == 'so2Value': #아황산가스
              # print(item.text)
              so2.append(item.text)
            elif item.tag == 'coValue': #일산화탄소
              # print(item.text)
              co.append(item.text)
            elif item.tag == 'o3Value': #오존
              # print(item.text)
              o3.append(item.text)
            elif item.tag == 'no2Value': #이산화질소
              # print(item.text)
              no2.append(item.text)
            elif item.tag == 'pm10Value': #미세먼지
              # print(item.text)
              pm10.append(item.text)
            elif item.tag == 'pm25Value': #미세먼지
              # print(item.text)
              pm25.append(item.text)

    num = len(dtime)
 
    seq = make_format(dtime[0])
    cnt = exec_select(conn,seq,sidoName)

    if cnt:
      print('overlap seq: ',dtime[0],sidoName)
    else:
      print('does not overlap seq: ',dtime[0],sidoName)
      for i in range(0,num):
        exec_insert(conn,seq,dtime[i],sidoName,area[i],so2[i],co[i],o3[i],no2[i],pm10[i],pm25[i])

    conn.close()

  except Exception as e:
    with open(ERR_URI,'a') as file:
      file.write('{} You got an error: {}\n'.format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),str(e)))

In [539]:
area = {
    1:['경기',31],
    2:['강원',17],
    3:['충북',11],
    4:['서울',25],
    5:['부산',16],
    6:['대구',10],
    7:['인천',9],
    8:['광주',5],
    9:['대전',5],
    10:['울산',5],
    11:['충남',15],
    12:['전북',14],
    13:['전남',21],
    14:['경북',14],
    15:['경남',18],
    16:['제주',2],
    17:['세종',1]
    }

numOfRows = area.get(15)[1] #지역 수
sidoName = area.get(15)[0] #경남

params = {'pageNo': 1, 'numOfRows': numOfRows, 'sidoName': sidoName, 'searchCondition': 'HOUR'}

resp = requests.get(URL+'?serviceKey='+change_keyword(SERVICEKEY), params=params)

# print(resp.text)

with open(RST_URI,'w') as file:
  file.write(resp.text)

data_collection(sidoName, RST_URI)

select   count(*) as cnt   from colabdust   where seq='202008121600' and area01='경남'
does not overlap seq:  2020-08-12 16:00 경남
insert into colabdust (seq,mdate,area01,area02,so2,co,o3,no2,pm10,pm25)   values ('202008121600','2020-08-12 16:00','경남','거제시','0.003','0.3','0.026','0.007','24','10')
insert into colabdust (seq,mdate,area01,area02,so2,co,o3,no2,pm10,pm25)   values ('202008121600','2020-08-12 16:00','경남','거창군','0.002','0.2','0.030','0.006','17','7')
insert into colabdust (seq,mdate,area01,area02,so2,co,o3,no2,pm10,pm25)   values ('202008121600','2020-08-12 16:00','경남','고성군','0.006','0.3','0.029','0.007','24','6')
insert into colabdust (seq,mdate,area01,area02,so2,co,o3,no2,pm10,pm25)   values ('202008121600','2020-08-12 16:00','경남','김해시','0.002','0.2','0.016','0.011','11','3')
insert into colabdust (seq,mdate,area01,area02,so2,co,o3,no2,pm10,pm25)   values ('202008121600','2020-08-12 16:00','경남','남해군','0.003','-','0.028','0.004','19','5')
insert into colabdust (seq,mdate,area0